# Pre Processing
---
The following headings give an overview of the preprocessing done in this notebook. 
### Cleaning
- We clean and rename some the columns and also change data types of some columns.
- We also drop the subjects which were excluded from the DREAM challenge.

### Transforming Dilution Column
- The dilution column is a string and as is it does not tell us the information we need namely is this observation of a higher or lower dilution - as each molecule is only presented to subjects at two dilutions - but these two dilutions vary greatly accross molecules.
- We end up with a `1` denoting a high concentration and a `0` denoting a low concentration.
- This transformation is nessecary to perform before our calculating mean responses and is nessecary to include otherwise our algorithms will treat the high dilution and the lower dilution as the same observation bacause they are identical otherwise accross all other descriptive features. 
- We can perform this calculation before splitting our test data because each value for dilution (0=lower, 1=higer) is assigned based on CID which is exactly how our train/test split is made. Thus our training data never interacts with out hidden test data.


### Calculating Mean Responses
- The best predictions made in the literature for population level responses used the actual means of responses (respective of the molecule/dilution in question) while masking NaN values in the calculation.
> "For subchallenge 2 I simply masked all NaNs for each subject and computed means and sigmas across subjects, ignoring the masked values. This made the most sense because the means and sigmas in the ground truth data also ignore NaNs." - *Team IKW*
- This approach avoids the imputation problem I have been dealing with in that all target columns in the raw dataset contain a majority of NaN values.
- See note below detailing why this is not an example of data leakage.

### Combining Datasets
- Using the CID which is common to all datasets we create two new datasets by combining the molecular descriptors (descriptive features) with both the raw & mean responses.
- We then persist these full datasest before applying our training test-split.

### Hide Training Set
- Before any transformation, scaling or imputation takes place we must split our data into a hidden test set.

### Persist Datasets
- Persist the following dataset:
- `raw_data_full` - for visualisation and analysis.
- `mean_data_full` - for visualisation and analysis.
- `raw_training_set` - training and evaluating models.
- `mean_training_set` - training and evaluating models. 
- `raw_test_set` - hidden testing set for final evaluation.
- `mean_test_set` - hidden testing set for final evaluation.


## Imports

In [1]:
# Math/Data Libraries
import scipy
import numpy as np
import pandas as pd

# Visalisation
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
sb.set_style('whitegrid')

# I/O
import json
import xlrd

# Jupyter/IPython Utility
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Paths

In [2]:
path_to_transformed_data = '../../data/transformed/'
path_to_data = '../../data/'
molecular_descriptors_file = 'dream/molecular_descriptors_data.txt'
perceptual_responses_file = 'dream/perceptual_descriptors.xlsx'
test_set_file = 'dream/CID_testset.txt'

## Reading Data

In [3]:
# Read in perceptual dataset from 2014 Keller & Voshal Study [cite]
perceptual_responses = pd.read_excel(path_to_data
                                     + perceptual_responses_file,
                                     header=1, skiprows=1)

In [4]:
# Load the molecular descriptors 
molecular_descriptors = pd.read_csv(path_to_data
                                    + molecular_descriptors_file,
                                    sep='\t', header = 0)

## Viewing Data

In [5]:
perceptual_responses.head()
print("Shape of Dataframe => " + perceptual_responses.shape.__str__())

,C.A.S.,Catalogue #*,CID,Odor,Odor dilution,Subject # (this study),Subject # (DREAM challenge),Gender,"Race (""unknown"" indicates that the subject did not wish to specify)",Ethnicity,...,ACID,WARM,MUSKY,SWEATY,AMMONIA/URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
0,2257-09-2,W401404,16741,2-Phenylethyl isothiocyanate,"1/1,000",1,1.0,M,Black,Non-Hispanic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2257-09-2,W401404,16741,2-Phenylethyl isothiocyanate,"1/100,000",1,1.0,M,Black,Non-Hispanic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2442-10-6,W358207,17121,1-Octen-3-yl acetate,"1/1,000",1,1.0,M,Black,Non-Hispanic,...,NaN,NaN,NaN,NaN,NaN,63.0,NaN,NaN,NaN,NaN
3,2442-10-6,W358207,17121,1-Octen-3-yl acetate,"1/100,000",1,1.0,M,Black,Non-Hispanic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2530-10-1,W352705,520191,"3-Acetyl-2,5-dimethylthiophene","1/100,000",1,1.0,M,Black,Non-Hispanic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shape of Dataframe => (55000, 38)


In [6]:
molecular_descriptors.head()
print("Shape of Dataframe => " + molecular_descriptors.shape.__str__())

,CID,complexity from pubmed,MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,Psychotic-80,Psychotic-50,Hypertens-80,Hypertens-50,Hypnotic-80,Hypnotic-50,Neoplastic-80,Neoplastic-50,Infective-80,Infective-50
0,126,93.1,122.13,8.142,10.010,15.305,10.193,16.664,0.667,1.020,...,0,0,0,0,0,0,0,0,0,0
1,176,31.0,60.06,7.508,4.483,8.422,4.432,9.249,0.560,1.053,...,0,0,0,0,0,0,0,0,0,0
2,177,10.3,44.06,6.294,3.768,7.095,3.977,8.040,0.538,1.014,...,0,0,0,0,0,0,0,0,0,0
3,180,26.3,58.09,5.809,5.295,9.978,5.739,11.455,0.530,0.998,...,0,0,0,0,0,0,0,0,0,0
4,196,114.0,146.16,7.308,11.493,20.727,11.625,22.914,0.575,1.036,...,0,0,0,0,0,0,0,0,0,0


Shape of Dataframe => (476, 4870)


### Note the common column: CID
---
This is a Compound IDentifier number used by the PubChem database. This can be uses to assign the appropriate molecular descriptors to each training example. The CAS column and the Catalogue # column could also have been used for this purpose but will be dropped as both just serve as duplicate unique identifiers for the molecules.

## Cleaning Data
---
First we will just make the data easier to work with.

In [7]:
# Strip whitespace from column names
perceptual_responses = perceptual_responses.rename(columns=lambda x: x.strip())
molecular_descriptors = molecular_descriptors.rename(columns=lambda x: x.strip())

# Drop unneeded columns
perceptual_responses.drop(labels=["C.A.S.", "Catalogue #*", "VIAL #", "Subject # (this study)"], axis=1, inplace=True)

# Rename verbose columns for readability & easier access
perceptual_responses = perceptual_responses.rename(index=str, columns={
    "HOW STRONG IS THE SMELL?": "INTENSITY",
    "HOW PLEASANT IS THE SMELL?": "PLEASANTNESS",
    "CAN OR CAN'T SMELL?": "CANSMELL?",
    "CAN OR CAN'T SMELL": "SMELL?",
    "KNOW OR DON'T KNOW THE SMELL": "SMELLKNOWN?",
    "THE ODOR IS:": "ODORGUESS",
    "AMMONIA/URINOUS": "URINOUS",
    "Subject # (DREAM challenge)": "SUBJECT",
    "Race\n (\"unknown\" indicates\n that the subject did not\n wish to specify)": "Race",
    "HOW FAMILIAR IS THE SMELL?": "FAMILIARITY"})

# Drop all observations from subjects who were excluded from DREAM challenge
# due to objectively poor response patterns such as high variability on repeated samples
perceptual_responses = perceptual_responses[perceptual_responses.loc[:, "SUBJECT"].notna()]
# reduces observations from 55000 to 49000 (55 subjects to 49 subjects)

# convert from numpy.int64 & int to string so we can sucesfully merge dataframes later
molecular_descriptors.CID = molecular_descriptors.CID.apply(str)
perceptual_responses.CID = perceptual_responses.CID.apply(str)

In [8]:
perceptual_responses.head()

,CID,Odor,Odor dilution,SUBJECT,Gender,Race,Ethnicity,Age,SMELL?,SMELLKNOWN?,...,ACID,WARM,MUSKY,SWEATY,URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
0,16741,2-Phenylethyl isothiocyanate,"1/1,000",1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16741,2-Phenylethyl isothiocyanate,"1/100,000",1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17121,1-Octen-3-yl acetate,"1/1,000",1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,63.0,NaN,NaN,NaN,NaN
3,17121,1-Octen-3-yl acetate,"1/100,000",1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,520191,"3-Acetyl-2,5-dimethylthiophene","1/100,000",1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
molecular_descriptors.head()

,CID,complexity from pubmed,MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,Psychotic-80,Psychotic-50,Hypertens-80,Hypertens-50,Hypnotic-80,Hypnotic-50,Neoplastic-80,Neoplastic-50,Infective-80,Infective-50
0,126,93.1,122.13,8.142,10.010,15.305,10.193,16.664,0.667,1.020,...,0,0,0,0,0,0,0,0,0,0
1,176,31.0,60.06,7.508,4.483,8.422,4.432,9.249,0.560,1.053,...,0,0,0,0,0,0,0,0,0,0
2,177,10.3,44.06,6.294,3.768,7.095,3.977,8.040,0.538,1.014,...,0,0,0,0,0,0,0,0,0,0
3,180,26.3,58.09,5.809,5.295,9.978,5.739,11.455,0.530,0.998,...,0,0,0,0,0,0,0,0,0,0
4,196,114.0,146.16,7.308,11.493,20.727,11.625,22.914,0.575,1.036,...,0,0,0,0,0,0,0,0,0,0


## Converting Dilutions - String to an int 
---
### Still just cleaning
---
The dilutions are recorded as strings as one of the following:

- '1/10'
- '1/100'
- '1/1,000'
- '1/10,000'
- '1/100,000'

Each molecule in the dataset contains observations of two dilutions. The high concentration and another low concentration. The dilution or concentration of a compund is known to have an effect on the perceived scent. One study shows that.... {cite}. 

---
We want a column that has `1` for the high concentration and `0` for the low concentration samples.
-  To do so we convert the strings to ints that represent their length.
- Then the function `high_or_low_dilution` checks which is the smaller dilution of the two and returns True if `x` is smaller and false if `x` is larger.
- Then we assign `1` to the shorter string lengths(high concentration - `True`) and `0` to the smaller number (low concentration - `False`) accross each molecule.

In [10]:
# convert the strings to ints based on their length. 
perceptual_responses['Odor dilution'] = perceptual_responses['Odor dilution'].str.len()

# Here we define a function that returns a boolean array
high_or_low_dilution = lambda x: (x < x.max())
# Combined with the following groupby(CID) - this gives dilutions by molecule
grouped_by_cid = perceptual_responses.groupby('CID')
# assign true for the highest dilution and false for the lower dilution
perceptual_responses['dilution_bool'] = grouped_by_cid['Odor dilution'].apply(high_or_low_dilution)

# overwrite the original column with 1 for high dilution and 0 for low dilution
perceptual_responses['Odor dilution'] = perceptual_responses['dilution_bool'].apply(lambda x: 1 if x else 0)
# Drop the boolean column
perceptual_responses.drop('dilution_bool', axis=1, inplace=True)

In [11]:
# Exactly half of the observations are true and half are false. This is what we expect
perceptual_responses['Odor dilution'].value_counts()

1    24500
0    24500
Name: Odor dilution, dtype: int64

## Calculating population mean responses
---
The best models to predict population level means have actually used the mean responses accross all subjects as their target features. This reduces our number of observations by the number of subjects - i.e `49`. so we go from `49000` to `1000` samples. This tips the problem into being an ill posed problem such that now the number of observations is less than the number of features. We can amend this by feature selection or by careful choice of algorithm (many of which are inherantly selective of the features used for predictions and thus are suited to tackling such problems).


Later we will output both the raw responses as well as the population means of those responses.


In [12]:
perceptual_responses.head()

,CID,Odor,Odor dilution,SUBJECT,Gender,Race,Ethnicity,Age,SMELL?,SMELLKNOWN?,...,ACID,WARM,MUSKY,SWEATY,URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
0,16741,2-Phenylethyl isothiocyanate,1,1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16741,2-Phenylethyl isothiocyanate,0,1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17121,1-Octen-3-yl acetate,1,1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,63.0,NaN,NaN,NaN,NaN
3,17121,1-Octen-3-yl acetate,0,1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,520191,"3-Acetyl-2,5-dimethylthiophene",0,1.0,M,Black,Non-Hispanic,45,I smell something,I don't know what the odor is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# We want the target columns and the CID & Odor dilution (to caclulate mean respective to both)
target_indices = [i for i in range(11, 34)]
target_indices.append(0)
target_indices.append(2)

target_df = perceptual_responses.iloc[:, target_indices]

# Mean of all subject responses per chemical per dilution
population_mean_responses = target_df.groupby(['CID', 'Odor dilution']).mean()

# We have NaN where no one in the population rated the substance with a descriptor
# remove these nan with zero imputation as this is closest to ground truth in data
population_mean_responses.fillna(0, inplace=True)
# 
population_mean_responses = population_mean_responses.reset_index().set_index('CID')
population_mean_responses.head(10)
population_mean_responses.shape

,Odor dilution,INTENSITY,PLEASANTNESS,FAMILIARITY,EDIBLE,BAKERY,SWEET,FRUIT,FISH,GARLIC,...,ACID,WARM,MUSKY,SWEATY,URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
CID,,,,,,,,,,,,,,,,,,,,,
1001,0,25.961538,47.807692,38.692308,22.666667,50.000000,8.666667,0.000000,35.000000,49.000000,...,36.000000,18.000000,26.750000,34.000000,45.571429,23.666667,37.600000,52.000000,18.666667,35.333333
1001,1,65.404255,28.425532,47.191489,28.000000,2.000000,13.000000,33.000000,30.526316,43.800000,...,23.000000,22.142857,40.941176,22.533333,15.714286,34.615385,11.200000,16.666667,13.833333,41.000000
101010,0,21.761905,48.666667,24.476190,59.000000,0.000000,13.500000,1.000000,6.000000,22.000000,...,11.000000,26.250000,26.222222,13.600000,44.000000,12.333333,42.333333,35.666667,59.500000,12.400000
101010,1,67.630435,39.760870,53.043478,33.000000,9.400000,21.400000,28.642857,18.666667,27.333333,...,21.545455,38.125000,27.266667,17.750000,16.200000,31.285714,2.400000,10.333333,24.200000,29.611111
101604,0,40.650000,66.125000,55.000000,34.705882,20.166667,43.967742,33.315789,0.000000,0.000000,...,3.000000,15.500000,9.666667,1.000000,0.000000,47.500000,8.000000,14.000000,30.818182,32.500000
101604,1,56.022222,72.222222,56.866667,36.526316,28.909091,44.194444,36.586207,0.000000,0.000000,...,65.000000,40.000000,36.500000,58.000000,32.000000,43.000000,78.000000,39.500000,36.416667,19.666667
10285,0,22.444444,48.722222,35.611111,48.000000,26.000000,25.000000,27.000000,2.000000,55.000000,...,9.000000,19.500000,27.666667,12.666667,3.000000,4.000000,10.333333,16.000000,6.000000,21.800000
10285,1,40.266667,40.366667,33.666667,0.000000,0.000000,15.000000,0.000000,2.000000,0.000000,...,20.000000,29.200000,39.100000,5.500000,23.000000,29.800000,48.800000,19.333333,6.000000,29.111111
1030,0,11.235294,42.411765,30.529412,5.000000,38.000000,33.000000,2.000000,0.000000,4.500000,...,12.250000,19.750000,22.400000,1.000000,2.500000,1.000000,26.000000,26.000000,3.500000,19.857143


(960, 24)

In [14]:
population_mean_responses = population_mean_responses.reset_index().set_index('CID')

### Note on data leakage
---
I had to think carefuly about whether it was acceptable to calculate the mean in this way before hidding our test set. As we are grouping by CID and odor dilution before we calculate our mean responses the `mean()` function never sees any data outside of these groupings. So in summary this type of calculation is not an example of data leakage as our training/test data spit is defined by the CID also - meaning our mean response for each chemical/dilution pair is not affected by other chemicals in the dataset. Thus with respect to this calculation our training set has not interacted with our test set.

## Combine Data on CID
---
This assigns the appropriate molecular descriptors to each row in the perceptual response dataset.


In [15]:
%%time
# Assign the molecular descriptors to each subject observation based on corresponding CID number
raw_data = pd.merge(perceptual_responses, molecular_descriptors, on='CID')
mean_data = pd.merge(population_mean_responses, molecular_descriptors, on='CID')

In [16]:
# We lose a small amount of observations in the combination
perceptual_responses.shape
print('=>')
raw_data.shape
population_mean_responses.shape
print('=>')
mean_data.shape

(49000, 34)

=>


(48412, 4903)

(960, 24)

=>


(948, 4894)

## Persist Raw Data

In [17]:
# Persist the full raw data set of visualisation, analysis and ease of access
# binary zip file loads much faster than parsing an excel or csv file
raw_data.to_pickle(path_to_transformed_data + "raw_data_full.zip")
mean_data.to_pickle(path_to_transformed_data + 'mean_data_full.zip')

## Hidden Test Set
---
For comparrison against the teams in the DREAM challenge we can use the same training and testing set split that was specified for the final submission. We have a file which tells us the moelcules to use for testing - we will use this to split our raw data and our mean responses. We can further split this training set into a validation set for training while keeping the final test set hidden for final evaluation

In [18]:
# Read in file and create a list of strings which identify the testing molecules
test_mol_cid = list(pd.read_csv(path_to_data + test_set_file, header=None)[0])
test_mol_cid_str = [str(i) for i in test_mol_cid]

# Assign raw responses to training and test
raw_test_set = raw_data[raw_data.CID.isin(test_mol_cid_str)== True]
raw_training_set = raw_data[raw_data.CID.isin(test_mol_cid_str)== False]

# Assign mean responses to training and test
mean_test_set = mean_data[mean_data.CID.isin(test_mol_cid_str)== True]
mean_training_set = mean_data[mean_data.CID.isin(test_mol_cid_str)== False]

# raw_data and mean_data have the same number of unique CIDs after test split
print("Number of test molecules: " + len(raw_test_set.CID.unique()).__str__())
print("Number of training molecules: " + len(mean_training_set.CID.unique()).__str__() + '\n')

# but differing numbers of observations (rows) 
# (no. of mean observations) * 49 = (no. of raw observations)
print("Number of raw test observations: " + raw_test_set.shape[0].__str__())
print("Number of raw training observations: " + raw_training_set.shape[0].__str__() + '\n')
print("Number of mean test observations: " + mean_test_set.shape[0].__str__())
print("Number of mean training observations: " + mean_training_set.shape[0].__str__())


Number of test molecules: 69
Number of training molecules: 405

Number of raw test observations: 6762
Number of raw training observations: 41650

Number of mean test observations: 138
Number of mean training observations: 810


## Persist training/validation & hidden test sets

In [19]:
raw_training_set.to_pickle(path_to_transformed_data + "raw_training_set.zip")
raw_test_set.to_pickle(path_to_transformed_data + "raw_test_set.zip")
mean_training_set.to_pickle(path_to_transformed_data + 'mean_training_set.zip')
mean_test_set.to_pickle(path_to_transformed_data + 'mean_test_set.zip')

In [20]:
%ls $path_to_transformed_data

imputation_old/        mean_training_set.zip  raw_training_set.zip
mean_data_full.zip     raw_data_full.zip
mean_test_set.zip      raw_test_set.zip
